In [ ]:
# !ls
%cd /content/drive/My Drive/yoloface


%matplotlib inline
from matplotlib import pyplot as plt

from utils import *
import cv2
import numpy as np

def iou(box1, box2):
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2
    
    # Calculate the (yi1, xi1, yi2, xi2) 
    xi1 = max(box1_x1,box2_x1)
    yi1 = max(box1_y1,box2_y1)
    xi2 = min(box1_x2,box2_x2)
    yi2 = min(box1_y2,box2_y2)
    inter_width = xi2-xi1
    inter_height = yi2-yi1
    #inter area
    inter_area = max(inter_width,0) * max(inter_height,0)

    box1_area = (box1_x2-box1_x1) * (box1_y2-box1_y1)
    box2_area = (box2_x2-box2_x1) * (box2_y2-box2_y1)
    #union area
    union_area = (box1_area + box2_area) - inter_area
    
    iou = inter_area / union_area
    return iou

class Feather_Track(object):
  def __init__(self, max_age=1, iou_thresh =0.4):
    #Age denotes the expiring of a track
    self.max_age = max_age
    self.iou_thresh = iou_thresh
    self.p_track = []
    self.frame_count = 0
    self.id = 0
    
  def update(self, dets=np.empty((0, 5))):
    # print(dets)
    self.frame_count += 1
    # self.p_track[:]['age'] += 1
    updated_trk = []
    # if self.id==0 and dets.size != 0:
    if not self.p_track and dets.size != 0:
      #Initialise first detections as Previous Trackers
      self.p_track = [{'bbox':list(det),'id':k,'age':0,'frame_count':1} for k,det in enumerate(dets,1)]
      self.id = self.p_track[-1]['id']
    elif self.p_track and dets.size != 0:
      # print("####")
      # print(self.p_track)
      new_p = self.p_track
      for det in dets:
        try:
          best_match_trk = max(new_p,key=lambda x:iou(det,x['bbox']))
        except ValueError:
          print("################################### VALUE ERROR ####################################################\n"+"det: ",end=" ")
          print(det)
          print("prev trk: ",end=" ")
          print(self.p_track)
          continue
        # print(iou(det,best_match_trk['bbox']))
        if iou(det,best_match_trk['bbox']) >= self.iou_thresh :
          best_match_trk['bbox'] = det
          best_match_trk['frame_count'] += 1
          # print(best_match_trk)
          updated_trk.append(best_match_trk)
        else:
          #New non matched detection is assigned to new track
          self.id += 1
          new_trk = {'bbox':det,'id':self.id,'age':0,'frame_count':1}
          updated_trk.append(new_trk)
          if best_match_trk['age'] <= self.max_age:
            best_match_trk['age'] += 1
            updated_trk.append(best_match_trk)
        #removing the used best_match_trk from p_track
        # del dets[dets.index(best_match)]
        # print(self.p_track[self.p_track.index(best_match_trk)])
        # print("before:")
        # print(self.p_track)
        # print(best_match_trk)
        # print(self.p_track.index(best_match_trk))
        # del self.p_track[self.p_track.index(best_match_trk)]
        # self.p_track.remove(best_match_trk)
        self.p_track = list(filter(lambda x: x['id'] != best_match_trk['id'] and x['age'] < self.max_age, self.p_track)) 
        # print("after:")
        # print(self.p_track)        
      #increase age of previous tracks
      for trk in self.p_track:
        trk['age'] += 1
      self.p_track = self.p_track + updated_trk
    elif self.p_track and dets.size == 0:
      new_p = []
      for trk in self.p_track:
        trk['age'] += 1
        if(trk['age'] < self.max_age):
          new_p.append(trk)
      self.p_track = new_p
      return []
        
      
    # print("#######",end = " ")
    # print(self.p_track,end="#######")
    return self.p_track




In [ ]:
model_cfg = './cfg/yolov3-face.cfg'
model_weights = './model-weights/yolov3-wider_16000.weights'
# image = './samples/outside_000001.jpg'
video = './samples/trance.mp4'
output_dir = '/content/drive/My Drive/yoloface/outputs/'
IOU_THRESH = 0.35
MAX_AGE = 1
#Trackers(Here faces) with given frame count value is chosen for age and gender processing
FRAME_COUNT_THRESH = 10
# DETECT_INTERVAL = 2
FACE_HEIGHT = 70
FACE_WIDTH = 70
# print the arguments
print('----- info -----')
print('[i] The config file: ',model_cfg)
print('[i] The weights of model file: ',model_weights)
# print('[i] Path to image file: ',image)
print('[i] Path to video file: ',video)
print('[i] IOU Threshold value: ',IOU_THRESH)
print('[i] Max age value: ',MAX_AGE)
print('[i] Frame count threshold value: ',FRAME_COUNT_THRESH)
# print('[i] Detect interval: ',DETECT_INTERVAL)
print('[i] Minimum face height required: ',FACE_HEIGHT)
print('[i] Minimum face width required: ',FACE_WIDTH)


#Read config net
net = cv2.dnn.readNetFromDarknet(model_cfg,model_weights)
print('###########################################################\n')


#VIDEO
cap = cv2.VideoCapture(video)
output_file = video[:-4].rsplit('/')[-1] + '_yoloface.avi'
output_file = output_dir + output_file
video_writer = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'),cap.get(cv2.CAP_PROP_FPS), (
                                          round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                                          round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(cap.get(cv2.CAP_PROP_FPS))
#create instance of the multi object tracker
mot = Feather_Track(MAX_AGE,IOU_THRESH)

----- info -----
[i] The config file:  ./cfg/yolov3-face.cfg
[i] The weights of model file:  ./model-weights/yolov3-wider_16000.weights
[i] Path to video file:  ./samples/trance.mp4
[i] IOU Threshold value:  0.35
[i] Max age value:  1
[i] Frame count threshold value:  10
[i] Minimum face height required:  70
[i] Minimum face width required:  70
###########################################################

1280.0
544.0
24.0


In [ ]:
ageProto="./age_gender_net/deploy_age.prototxt"
ageModel="./age_gender_net/age_net.caffemodel"
genderProto="./age_gender_net/deploy_gender.prototxt"
genderModel="./age_gender_net/gender_net.caffemodel"
MODEL_MEAN_VALUES=(78.4263377603, 87.7689143744, 114.895847746)
ageList=['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList=['Male','Female']
id_dict = {}

In [ ]:
#Frame counter for detect interval
c=0
while True:
  has_frame, frame = cap.read()
  if not has_frame:
      print('[i] ==> Done processing!!!')
      #print('[i] ==> Output file is stored at', os.path.join(args.output_dir, output_file))
      break
  # Resize, Convert BGR to HSV
  # if ((IMG_HEIGHT, IMG_WIDTH) != frame.shape[0:2]):
  #     frame = cv2.resize(frame, dsize=(IMG_WIDTH, IMG_HEIGHT), fx=0, fy=0)
  # else:
  #     frame = frame
  # if(c % DETECT_INTERVAL==0):
  new_frame = frame.copy()
  blob = cv2.dnn.blobFromImage(frame, 1 / 255, (IMG_WIDTH, IMG_HEIGHT),[0, 0, 0], 1, crop=False)
  net.setInput(blob)
  # Runs the forward pass to get output of the output layers
  outs = net.forward(get_outputs_names(net))
  # Remove the bounding boxes with low confidence
  faces = post_process(frame, outs,CONF_THRESHOLD, NMS_THRESHOLD)
  faces = np.array(faces) 
  if(faces.shape[0]>0):
    final_faces = faces[:,:4]
  else:
    final_faces = faces
  trackers = mot.update(final_faces)
  # print("LOG: Final faces = ",end=" ")
  # print(final_faces)
  # print("$$$$$$$$$$$$$ ") 
  # print(trackers)
  # print("&&&&&&&&&&&&&&&&&&")
  print("#",end="")

  for trk in trackers:
    box = np.array(trk['bbox'])
    id = int(trk['id'])
    frame_count = trk['frame_count']
    d = box.astype(np.int32)
    (startX, startY, endX, endY) = (d[0],d[1],d[2],d[3])
    face_h = endY - startY
    face_w = endX - startX
    if(frame_count==FRAME_COUNT_THRESH and (face_h > FACE_HEIGHT and face_w > FACE_WIDTH)):
      crop_face = new_frame[startY:endY, startX:endX]
      print(crop_face.shape)

      ageNet=cv2.dnn.readNet(ageModel,ageProto)
      genderNet=cv2.dnn.readNet(genderModel,genderProto)
      blob=cv2.dnn.blobFromImage(crop_face, 1.0, (227,227), MODEL_MEAN_VALUES, swapRB=False)
      genderNet.setInput(blob)
      genderPreds=genderNet.forward()
      gender=genderList[genderPreds[0].argmax()]
      # print("gender:",gender)
      ageNet.setInput(blob)
      agePreds=ageNet.forward()
      age=ageList[agePreds[0].argmax()]
      # print("Age:",age)
      id_dict[id] = [gender,age]
       
      try:
        cv2.imwrite("{0}/{1}_{2}_{3}.jpg".format("facepics",id,gender,age),crop_face)
      except:
        print("#########################Image error####################")
        continue

    if(id in id_dict.keys()):
       cv2.putText(frame, 'ID:%d %s' %(id,gender), (d[0] - 10, d[1] - 10),cv2.FONT_HERSHEY_SIMPLEX,0.5,COLOR_BLUE, 2)
    else:
      cv2.putText(frame, 'ID:%d' %id, (d[0] - 10, d[1] - 10),cv2.FONT_HERSHEY_SIMPLEX,0.5,COLOR_BLUE, 2)
    
    cv2.rectangle(frame,(d[0],d[1]),(d[2],d[3]),COLOR_BLUE, 3)
    #Frame count display
    text = '{}: {}'.format("Frame Count: ",c)
    cv2.putText(frame, text, (10,20),cv2.FONT_HERSHEY_SIMPLEX, 0.7, COLOR_RED, 2)

  c+=1
  video_writer.write(frame.astype(np.uint8))



In [ ]:
cap.release()
video_writer.release()

In [ ]:
############################################################################
#END